#Loading RAW data from bronze Layer (CRM)

## Init


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

##Importing Customer Data

In [0]:
df=spark.table("workspace.bronze.crm_cust_info")
df.display()

###Cleaning up Customer data

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))
# df.display()

In [0]:
df=(
    df.withColumn("cst_firstname",expr("concat(coalesce(cst_firstname,''),' ',coalesce(cst_lastname,cst_firstname))"))
    .withColumn("cst_marital_status",expr("""
                                          case
                                          when UPPER(cst_marital_status)='S' then 'Single'
                                          when UPPER(cst_marital_status)='M' then 'Married'
                                          else 'n/a'
                                          end 
                                          """))
    .withColumn("cst_gndr",expr("""
                                  case 
                                  when UPPER(cst_gndr)='M' then 'Male'
                                  when UPPER(cst_gndr)='F' then 'Female'
                                  else 'n/a'
                                  end
                                  """))
    .withColumnRenamed("cst_firstname","cst_name")
    .withColumnRenamed("cst_gndr","cst_gender")
    .drop("cst_lastname")
)
# df.display()

### Removing Customer ids  

In [0]:
df=df.filter(col("cst_id").isNotNull())
# df.display()

###Renaming Columns

In [0]:
RENAME_COLUMNS={
    "cst_id":"customer_id",
    "cst_key":"customer_key",
    "cst_name":"customer_name",
    "cst_marital_status":"customer_marital_status",
    "cst_gender":"customer_gender",
    "cst_create_date":"customer_create_date"
}
for old_name,new_name in RENAME_COLUMNS.items():
    df=df.withColumnRenamed(old_name,new_name)
df.display()

###Loading Customer Data

In [0]:
(
  df.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable("workspace.silver.crm_cust_info")
)